# Image Conversion

In [1]:
from utils.nrrd2nii import nrrd2nii

nrrd2nii('data/uwa_aaa/Patient1/Segmentation/CT_30%.nrrd', 'data/uwa_aaa/Patient1/Segmentation/CT_30%.nii.gz')

# Check Image

In [19]:
import nibabel as nib
import numpy as np
from utils.check_image import check_image
from utils.nrrd2nii import nrrd2nii

AAAlabel = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_AAA_label.nrrd")
Bloodlabel = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_blood_label.nrrd")
crop_image = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd")
image = nrrd2nii("data/uwa_aaa/Patient2/Segmentation/CT_30%.nrrd")
# ad = nib.load("data/uwa_aaa/Patient1/segmentations/adrenal_gland_left.nii.gz")


In [23]:

check_image(
    [
        {"image": image, "title": "image", "is_label": False},
        {"image": t, "title": "crop_image", "is_label": False},
        # {"image": AAAlabel, "title": "label", "is_label": True},
        # {"image": Bloodlabel, "title": "label", "is_label": True},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=168), IntSlider(value=0, description…

# Combine labels from totalsegmentor

In [1]:
import os
import nibabel as nib
import numpy as np


combine_label = np.zeros((512, 512, 281))
for root, dirs, files in os.walk("data/uwa_aaa/Patient1/segmentations"):
    images = [file for file in files if file.endswith(".nii.gz")]
    for i, image in enumerate(images):
        image = nib.load(os.path.join(root, image))
        combine_label[image.get_fdata().astype(bool)] = i + 10

In [33]:
def get_image(index):
    for root, dirs, files in os.walk("data/uwa_aaa/Patient1/segmentations"):
        images = [file for file in files if file.endswith(".nii.gz")]
        return nib.load(root + "/" + images[index])

In [35]:
from utils.nii_transform import label_transform
image = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_30%.nrrd")
AAA_label = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_AAA_label.nrrd")
blood_label = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_blood_label.nrrd")
final_label = combine_label.copy()
final_label[np.logical_and((combine_label == 0), (image.get_fdata() > -500))] = 100
aaa_label = label_transform(
    AAA_label.get_fdata(),
    image.get_fdata().shape,
    offset_x=216,
    offset_y=243,
    offset_z=83,
    flip_x=True,
    flip_y=True,
)
blood_label = label_transform(
    blood_label.get_fdata(),
    image.get_fdata().shape,
    offset_x=216,
    offset_y=243,
    offset_z=83,
    flip_x=True,
    flip_y=True,
)
final_label[aaa_label > 0.5] = 90
final_label[blood_label> 0.5] = 80

In [43]:
cropped = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd").get_fdata()

from skimage import io, img_as_float
from skimage.restoration import denoise_nl_means
denoised_image = denoise_nl_means(cropped)


In [44]:
from utils.check_image import check_image
from utils.nrrd2nii import nrrd2nii
from utils.nii_transform import label_transform


check_image(
    [
        {
            "image": image,
            "title": "image",
            "is_label": False,
        },
        {
            "image": final_label,
            "title": "image",
            "is_label": True,
        },
        {
            "image": denoised_image,
            "title": "image",
            # "is_label": True,
        },
        # {
        #     "image": nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd"),
        #     "title": "crop_image",
        #     "is_label": False,
        # },
        # {
        #     "image": nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_blood_label.nrrd"),
        #     "title": "label",
        #     "is_label": True,
        # },
        # {"image": Bloodlabel, "title": "label", "is_label": True},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=280), IntSlider(value=0, description…

# Image Registration

In [3]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
from utils.nii_transform import nii_transform
from utils.nrrd2nii import nrrd2nii
from utils.check_image import check_image

In [4]:
ref = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_30%.nrrd")
flo = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd")

In [5]:
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatText, Checkbox, FloatSlider

from utils.nii_transform import label_transform


def register_image(moving, static):
    # Assume first image defines the dimensions
    num_slices = static.shape[-1]
    moving = moving.get_fdata()
    static = static.get_fdata()

    def display_slice(
        slice_idx,
        offset_x,
        offset_y,
        offset_z,
        zoom_x,
        zoom_y,
        zoom_z,
        flip_x,
        flip_y,
        flip_z,
        alpha,
    ):
        plt.figure(figsize=(8, 8))
        plt.axis("off")
        label = label_transform(
            moving,
            static.shape,
            offset_x,
            offset_y,
            offset_z,
            zoom_x,
            zoom_y,
            zoom_z,
            flip_x,
            flip_y,
            flip_z,
        )
        plt.imshow(static[..., slice_idx], cmap="gray")
        plt.imshow(label[..., slice_idx], cmap="gray", alpha=alpha)
        plt.suptitle(f"NIfTI Image Slice {slice_idx}")
        plt.show()

    interact(
        display_slice,
        slice_idx=IntSlider(min=0, max=num_slices - 1, step=1, value=0),
        offset_x=FloatText(value=0.0, description="Offset X:"),
        offset_y=FloatText(value=0.0, description="Offset Y:"),
        offset_z=FloatText(value=0.0, description="Offset Z:"),
        zoom_x=FloatText(value=1.0, description="Zoom X:"),
        zoom_y=FloatText(value=1.0, description="Zoom Y:"),
        zoom_z=FloatText(value=1.0, description="Zoom Z:"),
        flip_x=Checkbox(value=False, description="Flip X"),
        flip_y=Checkbox(value=False, description="Flip Y"),
        flip_z=Checkbox(value=False, description="Flip Z"),
        alpha=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5),
    )

In [8]:
from utils.nii_flip import nii_flip

check_image(
    [
        {"image": nii_flip(ref, [0, 1]), "title": "image"},
        {"image": flo, "title": "image"},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=280), IntSlider(value=0, description…

In [7]:
register_image(flo, ref)

interactive(children=(IntSlider(value=0, description='slice_idx', max=280), FloatText(value=0.0, description='…

In [31]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

In [44]:
from ipywidgets import IntSlider, HBox, interactive_output

a = IntSlider()
b = IntSlider()
c = IntSlider()
ui = HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [40]:
# https://pypi.org/project/torchreg/
import torch
from torchreg import AffineRegistration

reg = AffineRegistration(
    iterations=(3000, 100),
    learning_rate=0.1,
    with_rotation=False,
    with_zoom=False,
    with_shear=False,
)

In [41]:
f_flo = nii_transform(flo, lambda x: np.flip(x, [0, 1]))
moved_alice = reg(
    torch.tensor(f_flo.get_fdata())[None, None].cuda(),
    torch.tensor(ref.get_fdata())[None, None].cuda(),
)

Shape: [1, 1, 64, 64, 113]; Dissimiliarity: 420423.6677505692:  42%|████▏     | 1274/3000 [00:10<00:14, 119.92it/s] 


KeyboardInterrupt: 

In [37]:
nii_image = nib.Nifti1Image(moved_alice[0, 0].cpu().detach().numpy(), flo.affine)

In [38]:
nii_image.get_fdata().shape

(256, 256, 452)

In [39]:
from utils.check_image import check_image

check_image(
    [
        {
            "image": nii_image,
            "title": "image",
        }
    ]
)

interactive(children=(IntSlider(value=0, description='slice_idx', max=451), Output()), _dom_classes=('widget-i…

In [17]:
nib.save(ref, "temp/ref.nii.gz")
nib.save(
    nii_transform(flo, lambda x: np.flip(zoom(x, [0.7, 0.7, 1]), [0, 1])),
    "temp/flo.nii.gz",
)

In [18]:
check_image(
    [
        {
            "image": nii_transform(flo, lambda x: np.flip(zoom(x, [1, 1, 1]), [0, 1])),
            "title": "image",
        }
    ]
)

interactive(children=(IntSlider(value=0, description='slice_idx', max=125), Output()), _dom_classes=('widget-i…

In [19]:
check_image(
    [
        {
            "image": ref,
            "title": "image",
        }
    ]
)

interactive(children=(IntSlider(value=0, description='slice_idx', max=280), Output()), _dom_classes=('widget-i…

In [20]:
# https://pypi.org/project/niftyreg/
# http://cmictig.cs.ucl.ac.uk/wiki/index.php/Reg_aladin
import niftyreg

niftyreg.main(
    [
        "aladin",
        "-ref",
        "temp/ref.nii.gz",
        "-flo",
        "temp/flo.nii.gz",
        "-res",
        "temp/out.nii.gz",
        "-noSym",
        "-rigOnly",
    ]
)

[/home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin] 
[/home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin] Command line:
[/home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin] 	 /home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin -ref temp/ref.nii.gz -flo temp/flo.nii.gz -res temp/out.nii.gz -noSym -rigOnly
[/home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin] 
[/home/tommy/miniconda3/envs/seg_3.12/lib/python3.12/site-packages/niftyreg/bin/reg_aladin] OpenMP is used with 12 thread(s)
[Aladin] Parameters
[Aladin] Platform: cpu_platform
[Aladin] Reference image name: temp/ref.nii.gz
[Aladin] 	512x512x281 voxels
[Aladin] 	1x1x1 mm
[Aladin] Floating image name: temp/flo.nii.gz
[Aladin] 	77x73x126 voxels
[Aladin] 	1x1x1 mm
[Aladin] Maximum iteration number: 5
[Aladin] 	(10 during the first level)
[Aladi

In [7]:
from utils.check_image import check_image
from utils.nrrd2nii import nrrd2nii
from utils.nii_transform import nii_transform
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

ref = nrrd2nii("data/uwa_aaa/Patient6/Segmentation/CT_30%.nrrd")
out = nib.load("temp/out.nii.gz")

check_image(
    [
        {
            "image": ref,
            "title": "out",
            "overlay": nii_transform(out, lambda data: zoom(data, [0.5, 0.5, 2])),
        },
        {"image": ref, "title": "ref"},
        {
            "image": nii_transform(out, lambda data: zoom(data, [0.5, 0.5, 2])),
            "title": "out",
        },
    ]
)

interactive(children=(IntSlider(value=0, description='slice_idx', max=451), Output()), _dom_classes=('widget-i…

In [3]:
ref.get_fdata().shape

(256, 256, 452)

In [33]:
out.get_fdata().shape

(256, 256, 452)

In [34]:
ref.get_fdata().shape

(256, 256, 452)

In [9]:
flo.get_fdata().shape

(74, 78, 113)

In [ ]:
import deep